In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ensemble/weighted_ensemble.pkl
/kaggle/input/test-pss5e1/test_pss5e1.csv
/kaggle/input/ey-final-boss/__results__.html
/kaggle/input/ey-final-boss/ey_test_niche_ds.csv
/kaggle/input/ey-final-boss/submission_all_data_shp.csv
/kaggle/input/ey-final-boss/ey_train_niche_ds.csv
/kaggle/input/ey-final-boss/__notebook__.ipynb
/kaggle/input/ey-final-boss/__output__.json
/kaggle/input/ey-final-boss/custom.css
/kaggle/input/ey-final-boss/__results___files/__results___47_0.png
/kaggle/input/ey-final-boss/__results___files/__results___7_1.png
/kaggle/input/test-df-ey-open-science/Submission_template_UHI2025-v2.csv
/kaggle/input/ey-peak-0-979-score/ey_train_niche_ds_0.979.csv
/kaggle/input/ey-peak-0-979-score/ey_test_niche_ds_0.979.csv
/kaggle/input/playground-series-s5e1/sample_submission.csv
/kaggle/input/playground-series-s5e1/train.csv
/kaggle/input/playground-series-s5e1/test.csv
/kaggle/input/train-pss5e1/train_pss5e1.csv
/kaggle/input/uhi-index-data/Training_data_uhi_index_UHI20

In [2]:
!pip install autogluon.tabular[0]
# !pip install autogluon.timeseries[0]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 23.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [3]:
# train = pd.read_csv('/kaggle/input/train-pss5e1/train_pss5e1.csv')
# test = pd.read_csv('/kaggle/input/test-pss5e1/test_pss5e1.csv')
# train = train.dropna(subset='num_sold')

In [4]:
# train['time_id'] = train['country'] + '_' + train['product'] + '_' + train['store']
# test['time_id'] = test['country'] + '_' + test['product'] + '_' + test['store']

In [5]:
# train['date'] = pd.to_datetime(train['date'])
# test['date'] = pd.to_datetime(test['date'])

In [6]:
# target = 'num_sold'
# features = [col for col in train.columns if (col!='num_sold' and col!='date' and col!='id')]
# cols_to_drop = ['date', 'id']
# train.drop(columns=cols_to_drop, inplace=True)
# test.drop(columns=cols_to_drop, inplace=True)

In [7]:
# from scipy.stats import boxcox
# train['num_sold'], lmbda = boxcox(train['num_sold'])

# ey tabular

In [8]:
train = pd.read_csv('/kaggle/input/ey-peak-0-979-score/ey_train_niche_ds_0.979.csv')
test  = pd.read_csv('/kaggle/input/ey-peak-0-979-score/ey_test_niche_ds_0.979.csv')
train_org  = pd.read_csv('/kaggle/input/uhi-index-data/Training_data_uhi_index_UHI2025-v2.csv')
test_org   = pd.read_csv('/kaggle/input/test-df-ey-open-science/Submission_template_UHI2025-v2.csv')

In [9]:
train['UHI Index'] = train_org['UHI Index']
target = 'UHI Index'
features = [col for col in train.columns if col!='UHI Index']

# Main code 

In [10]:
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label=target, eval_metric='r2', problem_type='regression')

# tabular_df = TimeSeriesDataFrame(train)
# label = 'num_sold'
# features = [col for col in tabular_df.columns if col!='num_sold']
# aml = TimeSeriesPredictor(label=label, eval_metric='mean_absolute_percentage_error', problem_type='regression')

No path specified. Models will be saved in: "AutogluonModels/ag-20250214_222127"


In [11]:
import warnings
warnings.filterwarnings("ignore")  # Suppress all warnings
predictor.fit(
    train_data=train,
    time_limit=39600,
    presets='best_quality',
    num_bag_folds=10,
    hyperparameters={
        'GBM': {'tree_method': 'gpu_hist', 'device': 'gpu'},  # LightGBM
        'CAT': {'task_type': 'GPU'},  # CatBoost
        'XGB': {'tree_method': 'gpu_hist', 'gpu_id': 0},  # XGBoost
        'NN_TORCH': {},  # Neural networks with GPU
        'RF': {},         # Random Forest
        'XT': {},         # Extra Trees
        'LR': {},          # Linear Regression
        # You can add others, e.g., KNN if desired:
        'KNN': {}
    },
    verbosity=0,
    num_gpus=1
)

Default metric period is 5 because R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Default metric period is 5 because R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Default metric period is 5 because R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Default metric period is 5 because R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Default metric period is 5 because R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Default metric period is 5 because R2 is/are not implem

In [12]:
leaderboard = predictor.leaderboard()
leaderboard.to_csv('leaderboard.csv', index=False)

In [13]:
leaderboard

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.976781,r2,7.741875,4493.483769,0.001336,0.333348,3,True,17
1,LinearModel_BAG_L2,0.976773,r2,7.689247,4483.048164,0.107404,0.828909,2,True,15
2,WeightedEnsemble_L2,0.976217,r2,6.602856,363.729892,0.001299,0.168170,2,True,9
3,ExtraTrees_BAG_L1,0.975972,r2,0.444099,4.303341,0.444099,4.303341,1,True,5
4,LightGBM_BAG_L2,0.975562,r2,7.624569,4502.448296,0.042726,20.229042,2,True,10
5,ExtraTrees_BAG_L2,0.975457,r2,8.133215,4489.343341,0.551372,7.124086,2,True,13
6,XGBoost_BAG_L2,0.975453,r2,7.633135,4492.321511,0.051292,10.102257,2,True,14
7,CatBoost_BAG_L2,0.975029,r2,7.613669,4546.236345,0.031826,64.017091,2,True,12
8,RandomForest_BAG_L2,0.974747,r2,8.135152,4519.331805,0.553309,37.112550,2,True,11
9,NeuralNetTorch_BAG_L2,0.972902,r2,7.730273,4715.162481,0.148430,232.943227,2,True,16


In [14]:
predictions = predictor.predict(test)
predictions

0       0.963721
1       0.963761
2       0.962795
3       0.961600
4       0.959217
          ...   
1035    1.040894
1036    1.043606
1037    1.041471
1038    1.038707
1039    1.037321
Name: UHI Index, Length: 1040, dtype: float32

In [15]:
test_org['UHI Index'] = predictions
test_org.to_csv('autogluon_niche_sub.csv', index=False)

# pss5e1 submission process

In [16]:
# def inverse_boxcox(y_transformed, lmbda):
#     if lmbda==0:
#         return np.exp(y_transformed)
#     else:
#         return np.power(y_transformed*lmbda+1, 1/lmbda)
# # preds = inverse_quantile(preds)
# preds = inverse_boxcox(predictions, lmbda)

In [17]:
# sample_test = pd.read_csv('/kaggle/input/playground-series-s5e1/test.csv')
# idx = sample_test['id']

In [18]:
# predictions_df = pd.DataFrame({
#     'id':idx,
#     'num_sold':preds
# })
# predictions_df.to_csv('submission.csv', index=False)